In [6]:
import pyttsx3
import speech_recognition as sr
from gtts import gTTS
import requests
import random
import language_tool_python
from bardapi import Bard


In [37]:
import asyncio
import time
import os

In [35]:
# BARD_API_KEY = 'eQgrkVX-PXO8jbQUf-t3A5YqLFGg6BraM6XnRxD0cB1-NzIYkjDRwlZOA37ZXtSFqfbt2g.'
# BARD_API_ENDPOINT = 'https://api.botpress.io/bots/eAgrkVD0ILATEs80uZR9eHtldgECJ3xbxCztw4lj70NEyxW8afau6s1xpSusM0fg_pge-Q/mod/nlu'

In [38]:
os.environ['_BARD_API_KEY']='eQgrkVX-PXO8jbQUf-t3A5YqLFGg6BraM6XnRxD0cB1-NzIYkjDRwlZOA37ZXtSFqfbt2g.'

In [26]:
def get_completion(prompt):
  response = Bard().get_answer(prompt)['content']
  # await asyncio.sleep(7)
  return response

In [39]:
get_completion("how to learn python")

'Learning Python can be an exciting and rewarding journey. It\'s a versatile language that can be used for a variety of purposes, from data science and web development to machine learning and automation.\n\nHere are some steps you can take to get started with learning Python:\n\n**1. Understand why you want to learn Python.**\n\nWhat are your goals? Do you want to use Python for work, personal projects, or just for fun? Knowing your motivation will help you stay focused and choose the right learning resources.\n\n**2. Set realistic expectations.**\n\nLearning Python takes time and effort. Don\'t expect to become an expert overnight. Be patient with yourself and celebrate your progress along the way.\n\n**3. Choose the right learning resources.**\n\nThere are many great resources available for learning Python, both online and offline. Some popular options include:\n\n* **Online courses:** Platforms like Coursera, edX, and Udemy offer a variety of Python courses for all levels.\n* **Inte

In [21]:
def display_and_store_response(bard_response):
    print(bard_response)
    with open('data.txt', 'a') as f:
        f.write("%s\n" % bard_response)

In [3]:
def text_to_speech(text):
    engine = pyttsx3.init()
    voices = engine.getProperty("voices")
    engine.setProperty("voice", voices[1].id)
    rate = engine.getProperty("rate")
    engine.setProperty("rate", 150)
    engine.say(text)
    engine.runAndWait()


In [4]:
def speech_to_text():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Please start speaking...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

        try:
            print("Recognizing ...")
            text = recognizer.recognize_google(audio, language="en-in")
            return text
        except Exception as e:
            print("Sorry, I didn't understand")
            return "Sorry, I didn't understand"


In [5]:

def get_random_question():
    questions = [
        "Tell me about yourself.",
        "What are your strengths?",
        "Can you describe a challenging situation you've faced at work?",
        "Where do you see yourself in five years?",
        "What is your greatest achievement?",
        "How do you handle stress or pressure?",
        "What is your approach to teamwork?",
        "Tell me about a time when you had to overcome a setback.",
    ]
    return random.choice(questions)


In [41]:

def send_request_to_bard(input_text, question):
    # headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {BARD_API_KEY}'}
    # data = {'text': input_text}

    try:
        # response = requests.post(BARD_API_ENDPOINT, headers=headers, json=data, timeout=10)  # Set a timeout (in seconds)
        # response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)
        # return response.json().get('text', '')
          prompt =f"""
  Your task is to act as a Text Analyzer.
  I'll give you text which is basically the answers given by a person for the questions asked. 
  And your job is to analyse the answer for that question and tell how accurate his answer was and how many grammar mistakes he had.
  Don't be conversational. I need a plain 10-40 word answer for this.
  Question and the Answer is shared below, delimited with triple backticks:
  ```{question}```,
  ```{input_text}```
  
  """
          return get_completion(prompt)
        
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return "Error: Unable to connect to the server. Please try again later."


In [ ]:
send_request_to_bard("My strengths includes learning new technology quickly and solving the problems based on it","What are your strengths?")

In [9]:
def analyze_english(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    return matches

In [42]:
def interview_chatbot():
    text_to_speech("Hello! I am your interview assistant. How can I help you today?")
    user_responses = []
    length = 4
    i = 0 
    while i<length:
        question = get_random_question()
        text_to_speech(question)

        user_input = speech_to_text()
        user_responses.append(user_input)

        if "exit" in user_input.lower():
            text_to_speech("Goodbye! Have a great day.")
            break

        text_to_speech("Please wait while I process your response.")
        bard_response = send_request_to_bard(user_input, question)
        # time.sleep(7)
        # Display BARD's response to the user and store it for later use
        display_and_store_response(bard_response)
        text_to_speech(bard_response)
        i += 1

    # Analyze English in user responses and generate a report
    report = "\n\n---- English Communication Report Card ----\n"
    for i, response in enumerate(user_responses, start=1):
        matches = analyze_english(response)
        if matches:
            report += f"\nResponse {i}:\n"
            for match in matches:
                report += f"- {match}\n"
        else:
            report += f"\nResponse {i}: No English issues found.\n"

    # Save the report to a file or print it
    with open("english_report.txt", "w") as file:
        file.write(report)

if __name__ == "__main__":
    interview_chatbot()


Please start speaking...
Recognizing ...
Answer lacks depth and specificity. Mentions a tool ("Truecaller") but doesn't explain its role in teamwork approach. Accuracy unclear, grammar error ("with all the people") present. (29 words)

Please start speaking...
Recognizing ...
Accuracy: Partially accurate. Highlights one strength (adaptability) but lacks detailed examples or other strengths. Grammar: 1 error ("implement all the way"). Suggestion: "My strengths include quickly grasping new technologies and explaining them effectively to others, guiding them through implementation."

Please start speaking...
Recognizing ...
Answer lacks details on specific teamwork skills ("used to" suggests past experience not current approach). Grammatically incorrect ("used to handle" should be "used to handling"). Accuracy unclear without context of expected answer. Consider revising for clarity and specificity.


Please start speaking...
Recognizing ...
Answer accuracy: Low. Mentions learning Python 